In [1]:
import json
from dotenv import load_dotenv
from griptape.memory.structure import ConversationMemory
from griptape.memory.tool import BlobToolMemory
from griptape.tasks import PromptTask, ToolkitTask
from griptape.drivers import OpenAiChatPromptDriver
from griptape.tasks import PromptTask
from griptape.structures import Pipeline

from openai_dm.tools import CharacterSheetUpdater
from openai_dm.character_sheet import Character

load_dotenv()

sheet_memory = BlobToolMemory()
character_sheet_updater = CharacterSheetUpdater(
    character_sheet=Character(),
    output_memory={"update_race": [sheet_memory]},
)
print(character_sheet_updater.character_sheet.race)
print(character_sheet_updater.character_sheet.final_ability_scores)


driver = OpenAiChatPromptDriver(
    model="gpt-4",
    max_tokens=500 # you can experiment with the number of tokens
)

conv_memory = ConversationMemory()
pipeline = Pipeline(
    memory=conv_memory,
)

None
AbilityScores(strength=8, dexterity=8, constitution=8, intelligence=8, wisdom=8, charisma=8)


In [2]:
pipeline.add_tasks(
    PromptTask('''
        The user said {{ args[0] }}. Respond with a json of the ability score bonuses for the chosen race.
        Example 1: I'll be a halfling
        {"strength": 0, "constitution": 0, "dexterity": 2, "intelligence": 0, "wisdom": 0, "charisma": 0}
        Example 2: I want to play as a human.
        {"strength": 1, "constitution": 1, "dexterity": 1, "intelligence": 1, "wisdom": 1, "charisma": 1}''',
        prompt_driver=driver,
    ),
    ToolkitTask(
        '''Update the character sheet with the new_race_chosen in {{ args[0] }}
            racial_bonuses = {{ parent_output }}
        ''', 
        tools = [character_sheet_updater],
        prompt_driver=driver,
    ),
)
output = pipeline.run("I'll be a tiefling")

[09/30/23 23:27:13] INFO     PromptTask 46b8be69b7ff4c11a17ca5db3ca35eea                                           
                             Input:                                                                                
                                     The user said I'll be a tiefling. Respond with a json of the ability score    
                             bonuses for the chosen race.                                                          
                                     Example 1: I'll be a halfling                                                 
                                     {"strength": 0, "constitution": 0, "dexterity": 2, "intelligence": 0,         
                             "wisdom": 0, "charisma": 0}                                                           
                                     Example 2: I want to play as a human.                                         
                                     {"strength": 1, "constitution": 1, "dexterity": 1, "intelligence": 1,         
                             "wisdom": 1, "charisma": 1}

[09/30/23 23:27:18] INFO     PromptTask 46b8be69b7ff4c11a17ca5db3ca35eea                                           
                             Output: {"strength": 0, "constitution": 0, "dexterity": 0, "intelligence": 1,         
                             "wisdom": 0, "charisma": 2}

                    INFO     ToolkitTask 89f7573757254e63b3cc26b9e1a6eb91                                          
                             Input: Update the character sheet with the new_race_chosen in I'll be a tiefling      
                                         racial_bonuses = {"strength": 0, "constitution": 0, "dexterity": 0,       
                             "intelligence": 1, "wisdom": 0, "charisma": 2}                                        
                    

[09/30/23 23:27:27] INFO     Subtask 7c7c504755ea4a04908c15c5d0d15c3e                                              
                             Thought: The user wants to update the character sheet with a new race, which is a     
                             Tiefling, and the racial bonuses associated with it. I can use the                    
                             CharacterSheetUpdater tool with the update_race activity to accomplish this.          
                                                                                                                   
                             Action: {"type": "tool", "name": "CharacterSheetUpdater", "activity": "update_race",  
                             "input": {"values": {"race": "tiefling", "racial_bonuses": "{\"strength\": 0,         
                             \"constitution\": 0, \"dexterity\": 0, \"intelligence\": 1, \"wisdom\": 0,            
                             \"charisma\": 2}"}}}

                    INFO     Subtask 7c7c504755ea4a04908c15c5d0d15c3e                                              
                             Observation: Character(name=None, class_=None, subclasse=None, level=1,               
                             race='tiefling', alignment=None, background=None, proficiency_bonus=2,                
                             base_ability_scores=AbilityScores(strength=8, dexterity=8, constitution=8,            
                             intelligence=8, wisdom=8, charisma=8), racial_ability_bonus=AbilityScores(strength=0, 
                             dexterity=0, constitution=0, intelligence=1, wisdom=0, charisma=2),                   
                             final_ability_scores=AbilityScores(strength=8, dexterity=8, constitution=8,           
                             intelligence=9, wisdom=8, charisma=10),                                               
                             saving_throw_proficiencies=SavingThrowProficiences(strength=False, dexterity=False,   
                             constitution=False, intelligence=False, wisdom=False, charisma=False),                
                             skill_proficiencies=SkillProficiences(acrobatics=False, animal_handling=False,        
                             arcana=False, athletics=False, deception=False, history=False, insight=False,         
                             intimidation=False, investigation=False, medicine=False, nature=False,                
                             perception=False, performance=False, persuasion=False, religion=False,                
                             sleight_of_hand=False, stealth=False, survival=False), AC=10, speed=30,               
                             hit_point_max=6)

[09/30/23 23:27:31] INFO     ToolkitTask 89f7573757254e63b3cc26b9e1a6eb91                                          
                             Output: The character sheet has been updated. The character is now a Tiefling with    
                             racial ability bonuses of +1 to Intelligence and +2 to Charisma. The final ability    
                             scores are now Strength: 8, Dexterity: 8, Constitution: 8, Intelligence: 9, Wisdom: 8,
                             and Charisma: 10.

In [3]:
character_sheet_updater.character_sheet.race

'tiefling'

In [4]:
character_sheet_updater.character_sheet.final_ability_scores

AbilityScores(strength=8, dexterity=8, constitution=8, intelligence=9, wisdom=8, charisma=10)